In [2]:
import pandas as pd

### Infos du csv

In [ ]:

df = pd.read_csv('data/earthquakes.csv')

print('Shape:', df.shape)

pd.set_option('display.max_columns', None)

df.head()

### Doublons ?

In [ ]:
# Check for strictly identical rows
num_duplicates = df.duplicated().sum()
print(f'Number of strictly identical rows: {num_duplicates}')

if num_duplicates > 0:
    print('Duplicate rows:')
    print(df[df.duplicated()])
else:
    print('No duplicate rows found.')

### Types de variables avant conversion

In [ ]:
df = pd.read_csv('data/earthquakes.csv')
df.dtypes

### Conversion des types des variables et création du parquet


In [ ]:
df = pd.read_csv('data/earthquakes.csv')

# tout convertir
df = df.convert_dtypes()

# puis les dates
date_columns = ['time', 'updated']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], utc=True, errors='coerce')

df.to_parquet('data/STEP01_earthquakes.parquet')

In [ ]:
df = pd.read_parquet('data/STEP01_earthquakes.parquet')
df.dtypes

### Vérification conversion des dates

In [ ]:
df = pd.read_csv('data/earthquakes.csv')

# Compter les valeurs vides dans 'time' et 'updated'
time_nulls = df['time'].isnull().sum()
updated_nulls = df['updated'].isnull().sum()

print(f"Nombre de valeurs vides dans 'time': {time_nulls}")
print(f"Nombre de valeurs vides dans 'updated': {updated_nulls}")

In [ ]:
df = pd.read_parquet('data/earthquakes.parquet')

# Compter les valeurs vides dans 'time' et 'updated'
time_nulls = df['time'].isnull().sum()
updated_nulls = df['updated'].isnull().sum()

print(f"Nombre de valeurs vides dans 'time': {time_nulls}")
print(f"Nombre de valeurs vides dans 'updated': {updated_nulls}")

### Les valeurs 0 de nst et magNst passent en valeurs vides

In [ ]:
df = pd.read_parquet('data/STEP01_earthquakes.parquet')

# Remplacer les 0 par NaN dans nst et magNst
df['nst'] = df['nst'].replace(0, pd.NA)
df['magNst'] = df['magNst'].replace(0, pd.NA)

# Sauvegarder le parquet mis à jour
df.to_parquet('data/STEP02_earthquakes.parquet')

### Vérification valeurs faussement différentes dans "place" (ex : Washington, washington, Washington DC)

In [ ]:
df = pd.read_parquet('data/STEP02_earthquakes.parquet')

# Nombre de valeurs uniques dans 'place' original
unique_original = df['place'].nunique()
print(f"Nombre de valeurs uniques dans 'place' original : {unique_original}")

# Compter les occurrences de chaque place
place_counts = df['place'].value_counts()

# Places qui apparaissent plus d'une fois
duplicates = place_counts[place_counts > 1]

print(f"Nombre de places qui apparaissent plus d'une fois : {len(duplicates)}")
print(f"Total d'occurrences pour ces places : {duplicates.sum()}")

if len(duplicates) > 0:
    print("Exemples de places avec leurs occurrences :")
    print(duplicates.head(10))

In [ ]:
import pandas as pd
import unicodedata
import re

def clean_place(place):
    if pd.isna(place):
        return place
    # Remove accents
    place = unicodedata.normalize('NFD', place).encode('ascii', 'ignore').decode('ascii')
    # Convert to lowercase
    place = place.lower()
    # Remove punctuation
    place = re.sub(r'[^\w\s]', '', place)
    # Remove extra spaces
    place = ' '.join(place.split())
    return place

df = pd.read_parquet('data/STEP02_earthquakes.parquet')

df['original_place'] = df['place']

# Apply cleaning to the 'place' column
df['place'] = df['place'].apply(clean_place)

# Count unique cleaned places
unique_clean = df['place'].nunique()
print(f"Nombre de valeurs uniques dans 'place' après nettoyage : {unique_clean}")

# Group by cleaned place and show examples of groupings
grouped = df.groupby('place')['original_place'].unique()
grouped_multiple = grouped[grouped.apply(len) > 1]

print("Exemples de lieux regroupés après nettoyage :")
for cleaned, originals in grouped_multiple.head(10).items():
    print(f"Nettoyé: '{cleaned}'")
    print(f"Originaux: {list(originals)}")
    print("---")

# Enregistrer le parquet avec la colonne 'place' nettoyée
df.to_parquet('data/STEP03_earthquakes.parquet')

In [ ]:
df = pd.read_parquet('data/STEP03_earthquakes.parquet')

# Nombre de valeurs uniques dans 'place' original
unique_original = df['place'].nunique()
print(f"Nombre de valeurs uniques dans 'place' original : {unique_original}")


### Ajout colonne magnitude uniformisé

In [12]:

def convert_magnitude(row):
    mag = row["mag"]
    mtype = row["magType"]

    if pd.isna(mag) or pd.isna(mtype):
        return np.nan

    mtype = str(mtype).lower()  # normalisation

    if mtype in ["ml", "mlg", "mlr"]:  
        return mag

    if mtype in ["md"]:
        return 0.85 * mag + 0.3

    if mtype in ["mw", "mwc", "mwb", "mwr", "mww"]:
        return (mag - 0.9) / 0.67

    if mtype in ["mc"]:
        return 0.85 * mag + 0.3

    if mtype in ["ma"]:
        return mag  

    return pd.NA


df = pd.read_parquet('data/STEP03_earthquakes.parquet')

# Appliquer la fonction de conversion
df['mag_uniform'] = df.apply(convert_magnitude, axis=1)

# Sauvegarder dans un nouveau parquet
df.to_parquet('data/STEP04_earthquakes.parquet')

print("Nouveau fichier 'data/STEP04_earthquakes.parquet' créé avec la colonne 'mag_uniform'.")


Nouveau fichier 'data/STEP04_earthquakes.parquet' créé avec la colonne 'mag_uniform'.


### Valeurs manquantes

In [13]:
df = pd.read_parquet('data/STEP04_earthquakes.parquet')

# Compter les valeurs manquantes par colonne
missing_values = df.isnull().sum()
print("Nombre de valeurs manquantes par colonne :")
print(missing_values)

Nombre de valeurs manquantes par colonne :
time                     0
latitude                 0
longitude                0
depth                    9
mag                 156449
magType             167407
nst                1211162
gap                 838549
dmin               1346742
rms                 211653
net                      0
id                       0
updated                  0
place                   11
type                     0
horizontalError    1531963
depthError          606685
magError           1781012
magNst             1096838
status                   1
locationSource           0
magSource                0
original_place          11
mag_uniform         704865
dtype: int64


### Création d'un petit parquet

In [14]:
df = pd.read_parquet('data/STEP04_earthquakes.parquet')

# Extraire les 10 premières lignes
df_lite = df.head(10)

# Sauvegarder en parquet lite
df_lite.to_parquet('data/earthquakes_lite.parquet')

print("Fichier 'data/earthquakes_lite.parquet' créé avec les 10 premières lignes.")

Fichier 'data/earthquakes_lite.parquet' créé avec les 10 premières lignes.
